In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from datetime import datetime, timedelta
from selenium.common.exceptions import NoSuchElementException
from requests import get, Session
from lxml import etree
from bs4 import BeautifulSoup
from numpy import random as rd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
df_teams = pd.read_csv('nba_teams.csv', sep=',')
df_players = pd.read_csv('nba_players.csv', sep=',')
df_games = pd.read_csv('nba_games-bask.csv', sep=',')

In [3]:
def nba(date, nav):
    url = f"https://www.nba.com/games?date={date.year}-{date.month:02}-{date.day:02}"
    nav.get(url)
    sleep(5)
    try:
        nav.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
    except NoSuchElementException:
        pass
    def tabela(cells, n, teams, ct, key):
        cont = 1
        players = []
        new_player = []
        for k in cells:
            if (k.text == "DNP - Coach's Decision") or (k.text == "TOTALS"):
                break
            if cont == 2:
                new_player.append(key)
                new_player.append(teams[ct])
            if cont <= n:
                new_player.append(k.text)
                cont += 1
            else:
                players.append(new_player)
                new_player = [k.text]
                cont = 2
        return players

    dict_games = {"game_id": [], "date": [], "player_id": [], "player_name": [], "player_team": [], 
             "against": [], "starter": [], "inicial_position": [], "min": [], "pts": []}
        
    def box(driver, xpath, cont = 0):
        r = 0
        try:
            a = nav.find_element(By.XPATH, xpath)
            a.click()
            sleep(4)
        except NoSuchElementException:
            if cont >= 1:
                nav.find_element(By.XPATH, xpath)
            else:
                r = driver.refresh()
                sleep(4)
                box(driver, xpath, 1)
    
    games = nav.find_elements(By.CLASS_NAME, 'GameCard_gcMain__q1lUW')
    cont_games = 1
    for i in range(len(games)):
        sleep(10)
        box(nav, f'//*[@id="__next"]/div[2]/div[2]/main/div[2]/div/div[1]/div[1]/div[{i+2}]/section[1]/ul/li[2]/a')
        
        # Times
        teams_name = nav.find_elements(By.CLASS_NAME, 'GameBoxscoreTeamHeader_gbt__b9B6g')
        teams_game = []
        for l in teams_name:
            teams_game.append(l.text)
        # Chave única para o jogo
        key = int(str(date.year) + str(date.month) + str(date.day) + str(cont_games))
        
        times = nav.find_elements(By.CLASS_NAME, 'GameBoxscore_gbTableSection__zTOUg')
        ct = 0
        for l in times:
            body = l.find_element(By.TAG_NAME, 'tbody')
            cells = body.find_elements(By.TAG_NAME, 'td')
            data_team = tabela(cells, 21, teams_game, ct, key)
            ct += 1
            
            # Atribuição ao dicionário
            for j in data_team:
                print(j)
                name = j[0].split('\n')
                inicial = None
                titular = False
                minutes = seconds = j[3].split(':')
                minutes = int(minutes[0])
                seconds = (int(seconds[1]))/60
                time_player = minutes + seconds
                if len(name) > 1:
                    inicial = name[1]
                    titular = True
                if j[2] == teams_game[0]:
                    ag = df_teams.query(f"full_name == '{(teams_game[1]).title()}'")
                    ag = (ag['team'].values)[0]
                    hm = df_teams.query(f"full_name == '{(teams_game[0]).title()}'")
                    hm = (hm['team'].values)[0]
                else:
                    ag = df_teams.query(f"full_name == '{(teams_game[0]).title()}'")
                    ag = (ag['team'].values)[0]
                    hm = df_teams.query(f"full_name == '{(teams_game[1]).title()}'")
                    hm = (hm['team'].values)[0]
                    
                jogador = df_players.query(f'player == "{name[0]}"')
                dict_games["game_id"].append(key)
                dict_games["date"].append(f"{str(date.year)}-{str(date.month)}-{str(date.day)}")
                try:
                    dict_games["player_id"].append((jogador['player_id'].values)[0])
                except IndexError:
                    dict_games["player_id"].append(None)
                dict_games["player_team"].append(hm)
                dict_games["player_name"].append(name[0])
                dict_games["against"].append(ag)
                dict_games["starter"].append(titular)
                dict_games["inicial_position"].append(inicial)
                dict_games["min"].append(time_player)
                dict_games["pts"].append(j[21])
            
            #print(data_team)
        nav.get(url)
        cont_games += 1
        sleep(8)
    
    y = nav.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/main/div[2]/div/div[1]/div[1]/div[2]').text
    if (games == []) and (y == 'NO GAMES SCHEDULED'):
        dict_games = {"game_id": [None], "date": [date], "player_id": [None], "player_name": [None], 
                      "player_team": [None], "against": [None], "starter": [None],
                      "inicial_position": [None], "min": [None], "pts": [None]}
        
        return dict_games
    else:
        return dict_games

In [4]:
def baskreference(date):

    header = {'User-Agent':
         "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
    
    # html inicial
    urlbase = 'https://www.basketball-reference.com'
    # acessa os jogos do dia
    url = f"https://www.basketball-reference.com/boxscores/?month={date.month}&day={date.day}&year={date.year}"
    # pega o html da pagina
    x = rd.randint(3, 11)
    sleep(x)
    session = Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    html = session.get(url, headers=header).content
    soup = BeautifulSoup(html, 'html.parser')
    # pega todos os links dentro da pagina
    href = soup.find_all('a', href=True)
    # separa somente os links para o boxscore dos jogos
    games = []
    for i in href:
        if f'/boxscores/{date.year}{date.month:02}{date.day:02}' in i['href']:
            games.append(i['href'])
    # remove os duplicados
    games = list(dict.fromkeys(games))
    cont_games = 1
    # dicionario vazio
    dict_games = {"game_id": [], "date": [], "player_id": [], "player_name": [], "player_team": [], 
                 "against": [], "starter": [], "min": [], "pts": []}
    for i in games:
        # acessa o boxscore do jogo
        url_game = urlbase+i
        # teams name
        x = rd.randint(3, 11)
        sleep(x)
        html_game = session.get(url_game, headers=header)
        soup_game = BeautifulSoup(html_game.content, 'html.parser')
        # transforma em etree e pega o nome do time
        dom = etree.HTML(str(soup_game))
        house_team = dom.xpath('//*[@id="content"]/div[2]/div[1]/div[1]/strong/a')[0].text
        away_team = dom.xpath('//*[@id="content"]/div[2]/div[2]/div[1]/strong/a')[0].text
        teams_game = [house_team, away_team]

        # Chave única para o jogo
        key = int(str(date.year) + str(date.month) + str(date.day) + str(cont_games))

        # Abreviações
        hm = df_teams.query(f"full_name == '{(teams_game[0])}'")
        ag = df_teams.query(f"full_name == '{(teams_game[1])}'")
        abrev = [(hm['also_team'].values)[0], (ag['also_team'].values)[0]]
        hm = (hm['team'].values)[0]
        ag = (ag['team'].values)[0]

        # Criando um objeto Beautiful Soup
        soup_table = BeautifulSoup(html_game.text, 'html.parser')
        tables = soup_table.find_all('table') 

        # Procurando pelas tabelas especificas
        cont_players = 1
        for j in abrev:
            table = soup_table.find('table', id=f'box-{j}-game-basic')
            print(f'box-{j}-game-basic')
            for l in table.tbody.find_all('tr'):
                columns = l.find_all('td')
                column_name = l.find_all('th')
                out = ['Did Not Play', 'Did Not Dress', 'Not With Team', 'Player Suspended', 'Inactive']
                if (columns != []) and (columns[0].text.strip() not in out):
                    print(columns[0].text.strip().split(':'))
                    minutes = seconds = columns[0].text.strip().split(':')
                    minutes = int(minutes[0])
                    seconds = (int(seconds[1]))/60
                    time_player = minutes + seconds

                    titular = True
                    if cont_players > 5:
                        titular = False
                    jogador = df_players.query(f'player == "{column_name[0].text.strip()}"')
                    dict_games["game_id"].append(key)
                    dict_games["date"].append(f"{str(date.year)}-{str(date.month)}-{str(date.day)}")
                    try:
                        dict_games["player_id"].append((jogador['player_id'].values)[0])
                    except IndexError:
                        dict_games["player_id"].append(None)
                    if j == abrev[0]:
                        dict_games["player_team"].append(hm)
                        dict_games["against"].append(ag)
                    else:
                        dict_games["player_team"].append(ag)
                        dict_games["against"].append(hm)
                    dict_games["player_name"].append(column_name[0].text.strip())

                    dict_games["starter"].append(titular)
                    dict_games["min"].append(time_player)
                    dict_games["pts"].append(int(columns[18].text.strip()))
                    print(date, hm, ag, column_name[0].text.strip(), int(columns[18].text.strip()))
                cont_players+=1

        cont_games+=1
    return dict_games


In [5]:
last_date = datetime.strptime(df_games['date'].iloc[-1], '%Y-%m-%d')
min_date = last_date + timedelta(days=2, hours=2)

if datetime.now() < min_date:
    print("Jogos Atualizados")
else:
    date = last_date + timedelta(days=1)
    #chrome_options = Options()
    #chrome_options.add_argument("-headless")
    #nav = webdriver.Chrome(options=chrome_options)
    while date.now() > min_date:
        print(date)
        try:
            dict_games = baskreference(date)
            new_df = pd.DataFrame(dict_games)
        except NoSuchElementException:
            dict_games = nba(date, nav)
            new_df = pd.DataFrame(dict_games)
            
        df = [df_games, new_df]
        df_games = pd.concat(df)
        min_date = min_date + timedelta(days=1)
        date = date + timedelta(days=1)
    #nav.quit()

2023-01-17 00:00:00
box-POR-game-basic
['36', '53']
2023-01-17 00:00:00 POR DEN Anfernee Simons 14
['34', '26']
2023-01-17 00:00:00 POR DEN Damian Lillard 44
['33', '26']
2023-01-17 00:00:00 POR DEN Jerami Grant 18
['29', '19']
2023-01-17 00:00:00 POR DEN Josh Hart 4
['29', '12']
2023-01-17 00:00:00 POR DEN Jusuf Nurkić 6
['17', '51']
2023-01-17 00:00:00 POR DEN Gary Payton II 0
['17', '35']
2023-01-17 00:00:00 POR DEN Shaedon Sharpe 13
['16', '26']
2023-01-17 00:00:00 POR DEN Drew Eubanks 6
['12', '02']
2023-01-17 00:00:00 POR DEN Nassir Little 2
['5', '44']
2023-01-17 00:00:00 POR DEN Jabari Walker 0
['2', '22']
2023-01-17 00:00:00 POR DEN Keon Johnson 4
['2', '22']
2023-01-17 00:00:00 POR DEN Trendon Watford 2
['2', '22']
2023-01-17 00:00:00 POR DEN Greg Brown III 0
box-DEN-game-basic
['34', '28']
2023-01-17 00:00:00 POR DEN Nikola Jokić 36
['32', '05']
2023-01-17 00:00:00 POR DEN Kentavious Caldwell-Pope 9
['31', '33']
2023-01-17 00:00:00 POR DEN Aaron Gordon 11
['30', '34']
2023-0

2023-01-18 00:00:00 CHA HOU Garrison Mathews 3
['10', '53']
2023-01-18 00:00:00 CHA HOU Josh Christopher 8
['8', '46']
2023-01-18 00:00:00 CHA HOU Usman Garuba 0
box-SAC-game-basic
['40', '08']
2023-01-18 00:00:00 SAC LAL Harrison Barnes 20
['39', '07']
2023-01-18 00:00:00 SAC LAL De'Aaron Fox 31
['36', '49']
2023-01-18 00:00:00 SAC LAL Keegan Murray 10
['32', '55']
2023-01-18 00:00:00 SAC LAL Kevin Huerter 18
['30', '51']
2023-01-18 00:00:00 SAC LAL Richaun Holmes 16
['20', '36']
2023-01-18 00:00:00 SAC LAL Trey Lyles 10
['17', '18']
2023-01-18 00:00:00 SAC LAL Chimezie Metu 6
['10', '38']
2023-01-18 00:00:00 SAC LAL Malik Monk 2
['10', '19']
2023-01-18 00:00:00 SAC LAL Davion Mitchell 3
['1', '19']
2023-01-18 00:00:00 SAC LAL KZ Okpala 0
box-LAL-game-basic
['36', '30']
2023-01-18 00:00:00 SAC LAL LeBron James 32
['26', '33']
2023-01-18 00:00:00 SAC LAL Patrick Beverley 3
['26', '15']
2023-01-18 00:00:00 SAC LAL Dennis Schröder 5
['22', '00']
2023-01-18 00:00:00 SAC LAL Troy Brown Jr.

box-CHI-game-basic
['37', '43']
2023-01-19 00:00:00 CHI DET DeMar DeRozan 26
['35', '29']
2023-01-19 00:00:00 CHI DET Zach LaVine 30
['35', '12']
2023-01-19 00:00:00 CHI DET Nikola Vučević 16
['30', '46']
2023-01-19 00:00:00 CHI DET Patrick Williams 10
['25', '15']
2023-01-19 00:00:00 CHI DET Ayo Dosunmu 13
['22', '25']
2023-01-19 00:00:00 CHI DET Alex Caruso 2
['17', '14']
2023-01-19 00:00:00 CHI DET Coby White 10
['16', '32']
2023-01-19 00:00:00 CHI DET Derrick Jones Jr. 12
['13', '54']
2023-01-19 00:00:00 CHI DET Goran Dragić 3
['1', '06']
2023-01-19 00:00:00 CHI DET Carlik Jones 2
['1', '06']
2023-01-19 00:00:00 CHI DET Dalen Terry 2
['1', '06']
2023-01-19 00:00:00 CHI DET Tony Bradley 0
['1', '06']
2023-01-19 00:00:00 CHI DET Malcolm Hill 0
['1', '06']
2023-01-19 00:00:00 CHI DET Marko Simonovic 0
box-DET-game-basic
['34', '53']
2023-01-19 00:00:00 CHI DET Bojan Bogdanović 25
['33', '10']
2023-01-19 00:00:00 CHI DET Jaden Ivey 16
['31', '18']
2023-01-19 00:00:00 CHI DET Killian Ha

box-IND-game-basic
['24', '54']
2023-01-20 00:00:00 IND DEN Andrew Nembhard 5
['24', '45']
2023-01-20 00:00:00 IND DEN Buddy Hield 9
['23', '39']
2023-01-20 00:00:00 IND DEN Aaron Nesmith 13
['23', '03']
2023-01-20 00:00:00 IND DEN Myles Turner 15
['15', '36']
2023-01-20 00:00:00 IND DEN Oshae Brissett 2
['31', '32']
2023-01-20 00:00:00 IND DEN Bennedict Mathurin 19
['19', '07']
2023-01-20 00:00:00 IND DEN T.J. McConnell 8
['16', '50']
2023-01-20 00:00:00 IND DEN Chris Duarte 18
['16', '17']
2023-01-20 00:00:00 IND DEN Terry Taylor 4
['15', '56']
2023-01-20 00:00:00 IND DEN Jalen Smith 9
['9', '01']
2023-01-20 00:00:00 IND DEN Isaiah Jackson 0
['7', '04']
2023-01-20 00:00:00 IND DEN Trevelin Queen 3
['6', '55']
2023-01-20 00:00:00 IND DEN Goga Bitadze 0
['5', '21']
2023-01-20 00:00:00 IND DEN James Johnson 6
box-DEN-game-basic
['31', '24']
2023-01-20 00:00:00 IND DEN Michael Porter Jr. 19
['30', '24']
2023-01-20 00:00:00 IND DEN Aaron Gordon 28
['29', '28']
2023-01-20 00:00:00 IND DEN 

box-MIL-game-basic
['37', '04']
2023-01-21 00:00:00 MIL CLE Jrue Holiday 28
['35', '30']
2023-01-21 00:00:00 MIL CLE Pat Connaughton 15
['34', '26']
2023-01-21 00:00:00 MIL CLE Brook Lopez 10
['30', '57']
2023-01-21 00:00:00 MIL CLE Bobby Portis 23
['27', '56']
2023-01-21 00:00:00 MIL CLE Grayson Allen 8
['28', '15']
2023-01-21 00:00:00 MIL CLE Joe Ingles 8
['17', '44']
2023-01-21 00:00:00 MIL CLE George Hill 5
['12', '09']
2023-01-21 00:00:00 MIL CLE Jevon Carter 2
['11', '50']
2023-01-21 00:00:00 MIL CLE Jordan Nwora 3
['4', '09']
2023-01-21 00:00:00 MIL CLE A.J. Green 0
box-CLE-game-basic
['37', '58']
2023-01-21 00:00:00 MIL CLE Evan Mobley 38
['35', '32']
2023-01-21 00:00:00 MIL CLE Darius Garland 21
['28', '33']
2023-01-21 00:00:00 MIL CLE Caris LeVert 13
['27', '20']
2023-01-21 00:00:00 MIL CLE Jarrett Allen 12
['24', '02']
2023-01-21 00:00:00 MIL CLE Isaac Okoro 9
['22', '19']
2023-01-21 00:00:00 MIL CLE Cedi Osman 13
['18', '47']
2023-01-21 00:00:00 MIL CLE Ricky Rubio 3
['18',

...

box-WAS-game-basic
['41', '15']
2023-01-25 00:00:00 WAS HOU Kyle Kuzma 33
['32', '50']
2023-01-25 00:00:00 WAS HOU Daniel Gafford 9
['29', '39']
2023-01-25 00:00:00 WAS HOU Bradley Beal 17
['25', '42']
2023-01-25 00:00:00 WAS HOU Delon Wright 5
['20', '11']
2023-01-25 00:00:00 WAS HOU Corey Kispert 7
['29', '57']
2023-01-25 00:00:00 WAS HOU Deni Avdija 10
['22', '03']
2023-01-25 00:00:00 WAS HOU Kendrick Nunn 12
['14', '34']
2023-01-25 00:00:00 WAS HOU Jordan Goodwin 5
['8', '58']
2023-01-25 00:00:00 WAS HOU Will Barton 8
['8', '27']
2023-01-25 00:00:00 WAS HOU Taj Gibson 2
['6', '23']
2023-01-25 00:00:00 WAS HOU Vernon Carey Jr. 0
box-HOU-game-basic
['38', '05']
2023-01-25 00:00:00 WAS HOU Jalen Green 19
['36', '42']
2023-01-25 00:00:00 WAS HOU Alperen Şengün 21
['33', '46']
2023-01-25 00:00:00 WAS HOU Kenyon Martin Jr. 16
['31', '12']
2023-01-25 00:00:00 WAS HOU Eric Gordon 9
['29', '26']
2023-01-25 00:00:00 WAS HOU Jabari Smith Jr. 6
['20', '59']
2023-01-25 00:00:00 WAS HOU Jae'Sean

box-UTA-game-basic
['34', '10']
2023-01-25 00:00:00 UTA POR Lauri Markkanen 24
['31', '51']
2023-01-25 00:00:00 UTA POR Mike Conley 8
['30', '29']
2023-01-25 00:00:00 UTA POR Jordan Clarkson 18
['29', '09']
2023-01-25 00:00:00 UTA POR Malik Beasley 10
['18', '22']
2023-01-25 00:00:00 UTA POR Walker Kessler 6
['27', '15']
2023-01-25 00:00:00 UTA POR Jarred Vanderbilt 12
['21', '46']
2023-01-25 00:00:00 UTA POR Collin Sexton 19
['17', '58']
2023-01-25 00:00:00 UTA POR Rudy Gay 11
['13', '53']
2023-01-25 00:00:00 UTA POR Talen Horton-Tucker 12
['11', '35']
2023-01-25 00:00:00 UTA POR Ochai Agbaji 0
['0', '53']
2023-01-25 00:00:00 UTA POR Udoka Azubuike 4
['0', '53']
2023-01-25 00:00:00 UTA POR Nickeil Alexander-Walker 0
['0', '53']
2023-01-25 00:00:00 UTA POR Leandro Bolmaro 0
['0', '53']
2023-01-25 00:00:00 UTA POR Simone Fontecchio 0
box-POR-game-basic
['42', '20']
2023-01-25 00:00:00 UTA POR Jerami Grant 19
['39', '33']
2023-01-25 00:00:00 UTA POR Damian Lillard 60
['38', '53']
2023-01

box-DAL-game-basic
['39', '41']
2023-01-26 00:00:00 DAL PHX Spencer Dinwiddie 36
['39', '40']
2023-01-26 00:00:00 DAL PHX Tim Hardaway Jr. 11
['36', '26']
2023-01-26 00:00:00 DAL PHX Dwight Powell 15
['34', '38']
2023-01-26 00:00:00 DAL PHX Dorian Finney-Smith 18
['3', '21']
2023-01-26 00:00:00 DAL PHX Luka Dončić 0
['32', '00']
2023-01-26 00:00:00 DAL PHX Josh Green 9
['30', '49']
2023-01-26 00:00:00 DAL PHX Reggie Bullock 10
['10', '09']
2023-01-26 00:00:00 DAL PHX Dāvis Bertāns 0
['8', '03']
2023-01-26 00:00:00 DAL PHX Frank Ntilikina 0
['5', '13']
2023-01-26 00:00:00 DAL PHX Jaden Hardy 0
box-PHO-game-basic
['42', '20']
2023-01-26 00:00:00 DAL PHX Mikal Bridges 16
['38', '22']
2023-01-26 00:00:00 DAL PHX Chris Paul 22
['32', '52']
2023-01-26 00:00:00 DAL PHX Cameron Johnson 22
['31', '05']
2023-01-26 00:00:00 DAL PHX Deandre Ayton 19
['28', '22']
2023-01-26 00:00:00 DAL PHX Torrey Craig 5
['18', '01']
2023-01-26 00:00:00 DAL PHX Damion Lee 0
['13', '22']
2023-01-26 00:00:00 DAL PHX

box-LAL-game-basic
['44', '13']
2023-01-28 00:00:00 LAL BOS LeBron James 41
['37', '34']
2023-01-28 00:00:00 LAL BOS Dennis Schröder 13
['36', '27']
2023-01-28 00:00:00 LAL BOS Patrick Beverley 15
['26', '49']
2023-01-28 00:00:00 LAL BOS Troy Brown Jr. 2
['19', '07']
2023-01-28 00:00:00 LAL BOS Thomas Bryant 3
['33', '53']
2023-01-28 00:00:00 LAL BOS Anthony Davis 16
['25', '19']
2023-01-28 00:00:00 LAL BOS Russell Westbrook 12
['23', '41']
2023-01-28 00:00:00 LAL BOS Rui Hachimura 6
['17', '57']
2023-01-28 00:00:00 LAL BOS Lonnie Walker IV 13
box-BOS-game-basic
['47', '20']
2023-01-28 00:00:00 LAL BOS Jayson Tatum 30
['38', '47']
2023-01-28 00:00:00 LAL BOS Grant Williams 9
['38', '37']
2023-01-28 00:00:00 LAL BOS Jaylen Brown 37
['37', '58']
2023-01-28 00:00:00 LAL BOS Al Horford 7
['35', '54']
2023-01-28 00:00:00 LAL BOS Derrick White 14
['39', '19']
2023-01-28 00:00:00 LAL BOS Malcolm Brogdon 26
['16', '21']
2023-01-28 00:00:00 LAL BOS Luke Kornet 2
['5', '30']
2023-01-28 00:00:00 

box-TOR-game-basic
['41', '07']
2023-01-28 00:00:00 TOR POR Fred VanVleet 11
['39', '08']
2023-01-28 00:00:00 TOR POR Pascal Siakam 24
['37', '44']
2023-01-28 00:00:00 TOR POR Scottie Barnes 18
['35', '39']
2023-01-28 00:00:00 TOR POR Gary Trent Jr. 19
['33', '25']
2023-01-28 00:00:00 TOR POR Precious Achiuwa 27
['17', '11']
2023-01-28 00:00:00 TOR POR Chris Boucher 10
['11', '38']
2023-01-28 00:00:00 TOR POR Juancho Hernangómez 3
['10', '23']
2023-01-28 00:00:00 TOR POR Malachi Flynn 0
['9', '09']
2023-01-28 00:00:00 TOR POR Thaddeus Young 7
['1', '32']
2023-01-28 00:00:00 TOR POR Ron Harper Jr. 2
['1', '32']
2023-01-28 00:00:00 TOR POR Christian Koloko 2
['1', '32']
2023-01-28 00:00:00 TOR POR Jeff Dowtin 0
box-POR-game-basic
['40', '46']
2023-01-28 00:00:00 TOR POR Jerami Grant 26
['40', '28']
2023-01-28 00:00:00 TOR POR Anfernee Simons 14
['36', '56']
2023-01-28 00:00:00 TOR POR Damian Lillard 30
['29', '31']
2023-01-28 00:00:00 TOR POR Nassir Little 8
['22', '07']
2023-01-28 00:00

In [6]:
df_games

,game_id,date,player_id,player_name,player_team,against,starter,min,pts
0,202110191.0,2021-10-19,1102.0,Stephen Curry,GSW,LAL,1.0,36.183333,21.0
1,202110191.0,2021-10-19,1168.0,Draymond Green,GSW,LAL,1.0,29.933333,6.0
2,202110191.0,2021-10-19,1485.0,Andrew Wiggins,GSW,LAL,1.0,25.733333,12.0
3,202110191.0,2021-10-19,1370.0,Jordan Poole,GSW,LAL,1.0,25.066667,20.0
4,202110191.0,2021-10-19,1283.0,Kevon Looney,GSW,LAL,1.0,15.466667,7.0
...,...,...,...,...,...,...,...,...,...
217,202312811.0,2023-1-28,1038.0,Malik Beasley,UTA,DAL,0.0,29.783333,19.0
218,202312811.0,2023-1-28,1458.0,Jarred Vanderbilt,UTA,DAL,0.0,22.450000,7.0
219,202312811.0,2023-1-28,1415.0,Collin Sexton,UTA,DAL,0.0,19.100000,12.0
220,202312811.0,2023-1-28,1003.0,Ochai Agbaji,UTA,DAL,0.0,17.633333,4.0


In [7]:
#df_games.to_csv("nba_games22-23.csv", index=False)
df_games.to_csv("nba_games-bask.csv", index=False)